In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Bidirectional, LSTM, Flatten, GlobalMaxPool1D, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import backend as K

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import seaborn as sns

2024-03-22 15:39:45.387821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 15:39:45.387877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 15:39:45.389070: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-22 15:39:45.396170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
train = pd.read_csv('train.csv') # My dataset

train # Display the first 5 rows of the train dataset

,sentence,emotion,pos_tags,named_entities,sentiment_scores,word2vec_embeddings
0,That game hurt.,sadness,"['DET', 'NOUN', 'VERB', 'PUNCT']",[],-0.5267,[ 0.05542755 0.07275391 0.0168457 0.033126...
1,>sexuality shouldn’t be a grouping category I...,happiness,"['SPACE', 'X', 'NOUN', 'AUX', 'PART', 'AUX', '...",[],0.0000,[-3.86629972e-02 2.17396129e-02 1.77334872e-...
2,"You do right, if you don't care then fuck 'em!",neutral,"['PRON', 'VERB', 'ADV', 'PUNCT', 'SCONJ', 'PRO...",[],0.1316,[ 1.75781250e-02 -2.71402989e-02 1.20768227e-...
3,Man I love reddit.,happiness,"['INTJ', 'PRON', 'VERB', 'NOUN', 'PUNCT']",[],0.6369,[ 0.17626953 -0.04703776 0.00406901 0.087239...
4,"[NAME] was nowhere near them, he was by the Fa...",neutral,"['X', 'NOUN', 'PUNCT', 'AUX', 'ADV', 'ADP', 'P...",['Falcon'],0.0000,[ 0.0814209 0.01818848 -0.05981445 0.065185...
...,...,...,...,...,...,...
629491,that was what i felt when i was finally accept...,happiness,"['PRON', 'AUX', 'PRON', 'PRON', 'VERB', 'SCONJ...",['a couple of years'],0.2732,[-0.04505539 0.01730728 -0.06948853 0.005584...
629492,i take every day as it comes i m just focussin...,fear,"['PRON', 'VERB', 'DET', 'NOUN', 'SCONJ', 'PRON...",['every day'],0.5859,[-0.03127872 0.13306023 0.06667911 0.076911...
629493,i just suddenly feel that everything was fake,sadness,"['PRON', 'ADV', 'ADV', 'VERB', 'SCONJ', 'PRON'...",[],-0.4767,[ 1.35253906e-01 9.81140137e-03 -5.22766113e-...
629494,im feeling more eager than ever to claw back w...,happiness,"['PRON', 'AUX', 'VERB', 'ADV', 'ADJ', 'SCONJ',...",['last week'],0.5095,[-0.00286865 0.11989049 -0.0135498 0.046508...


In [6]:
train['emotion'].value_counts()

emotion
happiness    258682
sadness      135826
anger         86327
neutral       55898
fear          51226
surprise      31486
disgust       10051
Name: count, dtype: int64

In [7]:
# Drop the first 200k happiness emotions in the emotion column
train = train.drop(train[train['emotion'] == 'happiness'].index[:100000])

# Reset the index
train = train.reset_index(drop=True)

# Display the first 5 rows
train.head()

,sentence,emotion,pos_tags,named_entities,sentiment_scores,word2vec_embeddings
0,That game hurt.,sadness,"['DET', 'NOUN', 'VERB', 'PUNCT']",[],-0.5267,[ 0.05542755 0.07275391 0.0168457 0.033126...
1,"You do right, if you don't care then fuck 'em!",neutral,"['PRON', 'VERB', 'ADV', 'PUNCT', 'SCONJ', 'PRO...",[],0.1316,[ 1.75781250e-02 -2.71402989e-02 1.20768227e-...
2,"[NAME] was nowhere near them, he was by the Fa...",neutral,"['X', 'NOUN', 'PUNCT', 'AUX', 'ADV', 'ADP', 'P...",['Falcon'],0.0000,[ 0.0814209 0.01818848 -0.05981445 0.065185...
3,"He isn't as big, but he's still quite popular....",anger,"['PRON', 'AUX', 'PART', 'ADV', 'ADJ', 'PUNCT',...",[],0.6297,[ 1.14733890e-01 4.03053276e-02 2.13256832e-...
4,"I have, and now that you mention it, I think t...",neutral,"['PRON', 'VERB', 'PUNCT', 'CCONJ', 'ADV', 'SCO...",[],0.0000,[ 0.09745789 0.00759125 0.07655334 0.148986...


In [8]:
# Amount of emotions in emotion column
train['emotion'].value_counts()

emotion
happiness    158682
sadness      135826
anger         86327
neutral       55898
fear          51226
surprise      31486
disgust       10051
Name: count, dtype: int64

In [9]:
test = pd.read_csv('test.csv', sep='\t')

test.head() #Display the first 5 rows of the test dataset

,id,sentence
0,0,Girls are happy when they get flowers
1,1,His jaw dropped in disbelief when he saw the p...
2,2,Sometimes the ugly stench makes me wanna throw...
3,3,The foul odor from the garbage bin was disgust...
4,4,"I can’t believe it, they lost the game in the ..."


In [26]:
# # Tokenize the sentences in the test dataset
test_sequences = tokenizer.texts_to_sequences(test['sentence'])

# # Pad the sequences to ensure uniform length
X_test = pad_sequences(test_sequences, maxlen=max_length, padding='post')

In [11]:
train_sentences = train['sentence'].values
train_labels = train['emotion'].values
test_sentences = test['sentence'].values

In [12]:
# Preprocess labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels_encoded)

In [13]:
# Tokenize sentences
tokenizer = Tokenizer(oov_token='<OOV>', num_words=10000000)
tokenizer.fit_on_texts(train_sentences)
tokenizer.fit_on_texts(test_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [14]:
# Pad sequences for training data
max_length = max(len(x) for x in train_sequences)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')

# Pad sequences for testing data using the same max_length
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

In [15]:
# Split the dataset into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(train_padded, train_labels_one_hot, test_size=0.2, random_state=42)

In [16]:
unique_emotions = train['emotion'].unique()
num_emotions = len(unique_emotions)
print("Unique emotions:", unique_emotions)
print("Number of unique emotions:", num_emotions)

Unique emotions: ['sadness' 'neutral' 'anger' 'surprise' 'disgust' 'fear' 'happiness']
Number of unique emotions: 7


In [17]:
# Define the parameters
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
embedding_dim = 64  # Dimensionality of the embedding layer
num_emotions = len(unique_emotions)  # Number of unique emotions

# Defining the RNN model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=400, input_length=178),  # Increased embedding dimension
    Bidirectional(LSTM(516, return_sequences=True)),  # Adjusted LSTM units
    # Dropout(0.5),  # Added dropout for regularization
    Dense(256, activation='relu'),
    GlobalMaxPool1D(),  # Global max pooling
    Dense(128, activation='relu'),  # Adjusted Dense layer units
    Dense(64, activation='relu'),
    # Dropout(0.5),  # Added dropout for regularization
    Dense(num_emotions, activation='softmax')  # Output layer
])

model.summary() # Display the model summary

2024-03-22 15:40:28.633911: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-03-22 15:40:28.634230: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14943 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 178, 400)          33470800  
                                                                 
 bidirectional (Bidirection  (None, 178, 1032)         3785376   
 al)                                                             
                                                                 
 dense (Dense)               (None, 178, 256)          264448    
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8

In [18]:
def f1_score(y_true, y_pred):
    # Calculate Precision and Recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())

    # Calculate F1 score
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [19]:
# Compiling the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=[f1_score])

In [20]:
# Training the model with early stopping
history = model.fit(X_train, y_train, 
                    epochs=10, 
                    batch_size=128, 
                    validation_data=(X_val, y_val),
                    verbose=1, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/10


2024-03-22 15:40:33.039791: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-03-22 15:40:33.640077: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ef2084b9370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-22 15:40:33.640116: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-22 15:40:33.645651: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711122033.777090  100453 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3310/3310 [==============================] - 341s 101ms/step - loss: 0.4293 - f1_score: 0.8263 - val_loss: 0.3586 - val_f1_score: 0.8575
Epoch 2/10
3310/3310 [==============================] - 298s 90ms/step - loss: 0.3343 - f1_score: 0.8671 - val_loss: 0.3472 - val_f1_score: 0.8607
Epoch 3/10
3310/3310 [==============================] - 286s 86ms/step - loss: 0.3045 - f1_score: 0.8765 - val_loss: 0.3547 - val_f1_score: 0.8634
Epoch 4/10
3310/3310 [==============================] - 203s 61ms/step - loss: 0.2810 - f1_score: 0.8824 - val_loss: 0.3687 - val_f1_score: 0.8605
Epoch 5/10
3310/3310 [==============================] - 204s 62ms/step - loss: 0.2610 - f1_score: 0.8870 - val_loss: 0.4005 - val_f1_score: 0.8584
Epoch 6/10
3310/3310 [==============================] - 199s 60ms/step - loss: 0.2450 - f1_score: 0.8901 - val_loss: 0.4301 - val_f1_score: 0.8601
Epoch 7/10
3310/3310 [==============================] - 198s 60ms/step - loss: 0.2314 - f1_score: 0.8928 - val_loss: 0.4439 - va

In [21]:
print(len(X_val), len(y_val))

105900 105900


In [29]:
#y_pred = model.predict(X_val) # For confusion matrix
y_pred = model.predict(test_padded) # For Kaggle submission

45/45 [==============================] - 0s 9ms/step


In [30]:
# Convert predictions to label indices
predicted_class_indices = np.argmax(y_pred, axis=1)
true_classes = np.argmax(y_val, axis=1)

# Use the LabelEncoder to decode the predictions to original labels
predicted_emotions = label_encoder.inverse_transform(predicted_class_indices)

In [31]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix
cm = confusion_matrix(true_classes, predicted_class_indices)

# Normalize the confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot the confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm_normalized, annot=True, fmt=".2%", cmap="Blues", xticklabels=unique_emotions, yticklabels=unique_emotions)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Emotion")
plt.ylabel("True Emotion")
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [105900, 1436]

In [32]:
# Add predictions to my test dataset
test['emotion'] = predicted_emotions

In [33]:
test.head()

,id,sentence,emotion
0,0,Girls are happy when they get flowers,neutral
1,1,His jaw dropped in disbelief when he saw the p...,neutral
2,2,Sometimes the ugly stench makes me wanna throw...,anger
3,3,The foul odor from the garbage bin was disgust...,disgust
4,4,"I can’t believe it, they lost the game in the ...",surprise


In [34]:
# Drop sentence column
test = test.drop(columns=['sentence'])

test.head()

,id,emotion
0,0,neutral
1,1,neutral
2,2,anger
3,3,disgust
4,4,surprise


In [35]:
test.to_csv('rnn_model_yuliia.csv', index=False)